In [1]:
import geopandas as gpd
import pandas as pd

## Open State data

In [2]:
file_path = "data/cb_2018_us_county_500k/cb_2018_us_county_500k.shp"
counties_gdf = gpd.read_file(file_path)
counties_gdf = counties_gdf[counties_gdf["STATEFP"] != "09"][["geometry", "AFFGEOID"]]
counties_gdf = counties_gdf.to_crs(4326)

#### Format for CT

In [3]:
ct_gdf = gpd.read_file("data/CT_Planning_Regions.geojson")
ct_gdf["AFFGEOID"] = "0500000US" + ct_gdf["PlanningRegionFIPS_GEOID"]

In [4]:
counties_gdf = pd.concat(
    [ct_gdf[["geometry", "AFFGEOID"]], counties_gdf], ignore_index=True
)

## Get Plumbing Percents

In [5]:
acs_home = pd.read_csv(
    "data/ACSDP5Y2023.DP04_2025-02-06T161358/ACSDP5Y2023.DP04-Data.csv",
    low_memory=False,
)

#### Get Column names

In [6]:
acs_column_names = pd.read_csv(
    "data/ACSDP5Y2023.DP04_2025-02-06T161358/ACSDP5Y2023.DP04-Column-Metadata.csv"
)

In [7]:
home_cols = list(
    acs_column_names[
        (acs_column_names["Column Name"] == "DP04_0073E")
        | (acs_column_names["Column Name"] == "DP04_0073PE")
    ]["Column Name"]
)

#### Back to Data

In [8]:
acs_plumbing_data_per_county_df = acs_home[["GEO_ID", *home_cols]]
acs_plumbing_data_per_county_df.columns = acs_plumbing_data_per_county_df.iloc[0]
acs_plumbing_data_per_county_df = acs_plumbing_data_per_county_df[1:]
acs_plumbing_data_per_county_df = acs_plumbing_data_per_county_df.rename(
    columns={
        "Estimate!!SELECTED CHARACTERISTICS!!Occupied housing units!!Lacking complete plumbing facilities": "LACKING_PLUMBING",
        "Percent!!SELECTED CHARACTERISTICS!!Occupied housing units!!Lacking complete plumbing facilities": "PERCENT",
        "Geography": "AFFGEOID",
    }
)

## Merge Data

In [10]:
acs_plumbing_data_per_county_gdf = counties_gdf.merge(
    acs_plumbing_data_per_county_df, on="AFFGEOID", how="left"
)

In [11]:
acs_plumbing_data_per_county_gdf["county_id"] = acs_plumbing_data_per_county_gdf[
    "AFFGEOID"
].str[9:]
acs_plumbing_data_per_county_gdf["state_id"] = acs_plumbing_data_per_county_gdf[
    "AFFGEOID"
].str[9:11]

In [14]:
acs_plumbing_data_per_county_gdf = acs_plumbing_data_per_county_gdf.fillna(0)
acs_plumbing_data_per_county_gdf["LACKING_PLUMBING"] = acs_plumbing_data_per_county_gdf[
    "LACKING_PLUMBING"
].astype(int)
acs_plumbing_data_per_county_gdf["PERCENT"] = acs_plumbing_data_per_county_gdf[
    "PERCENT"
].astype(float)

In [15]:
acs_plumbing_data_per_county_gdf = acs_plumbing_data_per_county_gdf.to_crs(9311)
acs_plumbing_data_per_county_gdf.to_file("data/plumbing_per_county.gpkg")

In [16]:
acs_plumbing_data_per_county_gdf.sort_values("PERCENT", ascending=False)

,geometry,AFFGEOID,LACKING_PLUMBING,PERCENT,county_id,state_id
1080,"POLYGON ((-2719625.094 3167492.219, -2718060.9...",0500000US02290,776,33.5,02290,02
737,"MULTIPOLYGON (((-3101902.04 2893602.272, -3101...",0500000US02050,1198,25.7,02050,02
490,"MULTIPOLYGON (((-2772662.37 3340094.061, -2772...",0500000US02158,461,24.4,02158,02
37,"MULTIPOLYGON (((-2582444.172 3292250.254, -258...",0500000US02180,624,22.4,02180,02
1619,"MULTIPOLYGON (((-2380396.888 3432550.411, -238...",0500000US02188,348,19.2,02188,02
...,...,...,...,...,...,...
3138,"POLYGON ((648823.527 -499123.855, 648871.836 -...",0500000US29103,0,0.0,29103,29
699,"POLYGON ((1512007.13 -643245.397, 1512065.233 ...",0500000US21159,0,0.0,21159,21
739,"POLYGON ((651237.224 -1197055.773, 651510.364 ...",0500000US05039,0,0.0,05039,05
758,"POLYGON ((-641882.931 -602213.471, -641644.668...",0500000US08097,0,0.0,08097,08
